<a href="https://colab.research.google.com/github/jorgemarquesferreirajunior/YOLO/blob/main/Treinamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training Model

##01- Ativar GPU e Acessar Google Drive

In [ ]:
import tensorflow as tf
GPU = tf.test.gpu_device_name()
print(GPU)

# Saida esperada: /device:GPU:0

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Saida esperada: Mounted at, /content/gdrive

##02- Baixar ferramentas

In [ ]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git

In [ ]:
cd /content/OIDv4_ToolKit

In [ ]:
!pip3 install -r /content/OIDv4_ToolKit/requirements.txt

In [ ]:
cd /content/

In [ ]:
!git clone https://github.com/jorgemarquesferreirajunior/converter_annotations_yolo.git

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
!cp /content/converter_annotations_yolo/converter_annotations.py /content/OIDv4_ToolKit

##03- Descompactar dataset do drive para a pasta OIDv4_ToolKit

In [ ]:
!unzip /content/gdrive/MyDrive/ColabNotebooks/CatDetector/dataset.zip -d /content/OIDv4_ToolKit/

In [ ]:
!mv  /content/OIDv4_ToolKit/content/OIDv4_ToolKit/OID /content/OIDv4_ToolKit/OID

In [ ]:
!rm -rf /content/OIDv4_ToolKit/content

##04- Configurar arquivo de classes da pasta OIDv4_ToolKit e executar o script para converter as anotações

In [ ]:
cd /content/OIDv4_ToolKit/

In [ ]:
!cat classes.txt

In [ ]:
!echo -e 'Cat' > classes.txt # Adicionar os nomes das classes desejadas

In [ ]:
!python converter_annotations.py

##05- Renomear as pastas que contem as imagens de teste e de treino

In [ ]:
# eh necessario renomear as pastas para que a compilacao da darknet funcione adequadamente
!mv /content/OIDv4_ToolKit/OID/Dataset/test/Cat /content/OIDv4_ToolKit/OID/Dataset/test/valid/
!mv /content/OIDv4_ToolKit/OID/Dataset/train/Cat /content/OIDv4_ToolKit/OID/Dataset/train/obj

##06- Copiar o arquivo yolov4.cfg para o drive , editar o arquivo copiado e salvar na pasta darknet/cfg:

In [ ]:
!cp /content/darknet/cfg/yolov4.cfg /content/darknet/cfg/yolov4_custom.cfg

In [ ]:
cd /content/converter_annotations_yolo

In [ ]:
!python -c "from utils import configcfg; configcfg('/content/darknet/cfg/yolov4_custom.cfg', 64, 1)"

In [ ]:
!cp /content/darknet/cfg/yolov4_custom.cfg /content/gdrive/MyDrive/ColabNotebooks/CatDetector/yolov4_custom.cfg

In [ ]:
!cp /content/gdrive/MyDrive/ColabNotebooks/CatDetector/yolov4_custom.cfg /content/darknet/cfg

In [ ]:
!cp /content/darknet/cfg/yolov4.cfg /content/darknet/cfg/yolov4_custom.cfg

##07- Configurar outros arquivos para a darknet, salvar no drive e copiar para a darknet/data

In [ ]:
cd /content/gdrive/MyDrive/ColabNotebooks/CatDetector/

In [ ]:
!touch obj.names
!touch obj.data

In [ ]:
!echo -e 'Cat' > obj.names

In [ ]:
!cat obj.names

In [ ]:
#Se preferir, pode fazer manualmente essas alteracoes no arquivo
!echo -e 'classes = 1\ntrain = /content/darknet/data/train.txt\nvalid = /content/darknet/data/test.txt\nnames = /content/darknet/data/obj.names\nbackup = /content/gdrive/MyDrive/ColabNotebooks/CatDetector/backupWeights/' > obj.data

In [ ]:
!cat obj.data

In [ ]:
!cp /content/darknet/data/obj.names /content/gdrive/MyDrive/ColabNotebooks/CatDetector/obj.names
!cp /content/darknet/data/obj.data /content/gdrive/MyDrive/ColabNotebooks/CatDetector/obj.data

##08- Copiar as imagens para a pasta darknet/data e gerar arquivos txt com os caminhos de todas as imagens

In [ ]:
!cp -r /content/OIDv4_ToolKit/OID/Dataset/test/valid /content/darknet/data
!cp -r /content/OIDv4_ToolKit/OID/Dataset/train/obj /content/darknet/data

In [ ]:
!rm /content/darknet/data/obj/Label/*
!rm /content/darknet/data/valid/Label/*

In [ ]:
!cp /content/converter_annotations_yolo/gerar_test.py /content/darknet/
!cp /content/converter_annotations_yolo/gerar_train.py /content/darknet/

In [ ]:
cd /content/darknet/

In [ ]:
!python gerar_test.py

In [ ]:
!python gerar_train.py

##09- Compilar a darknet

In [ ]:
cd /content/darknet

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
!make

##10- Baixar pesos pré-treinados (transfer learning)

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
ls

##11- Testar o funcionamento da darknet

In [ ]:
!./darknet detect cfg/yolov4.cfg yolov4.weights data/person.jpg

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

def mostrar(path):
    img = cv2.imread(path)
    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis("off")
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

def detectar(imagem):
  os.system(f"cd /content/darknet && ./darknet detector test data/obj.data cfg/yolov4_custom.cfg yolov4_custom_last.weights {imagem}")
  mostrar('predictions.jpg')

In [ ]:
mostrar('/content/darknet/predictions.jpg')

##12- Mão na massa: Inicio do treinamento - OBS: Quiantidade mínima de epocas = 2000 * numero de classes

In [ ]:
!cp /content/gdrive/MyDrive/ColabNotebooks/CatDetector/yolov4_custom_last.weights ./

In [ ]:
#na primeira vez inicie o treinamento a partir dos pesos pre-treinados baixados do github,
# os pesos recalculados sao gerados a cada 100 epocas
#se necessario interromper o treinamento antes do termino, voce deve retoma-lo a partir dos
# novos pesos calculados ate a ultima epoca antes da interrupcao

#iniciano o treinamento
!./darknet detector train data/obj.data cfg/yolov4_custom.cfg yolov4.weights yolov4.conv.137 -dont_show -map

#continuando a treinamento apos uma pausa
#!./darknet detector train data/obj.data cfg/yolov4_custom.cfg yolov4_custom_last.weights -dont_show -map

##13- Verificando o map (Mean Average Precision)

In [ ]:
# atentar-se para o resultado da variavel ap, quanto mais proximo de 100% melhor
!./darknet detector map data/obj.data cfg/yolov4_custom.cfg yolov4_custom_last.weights

##11- Testar o funcionamento da darknet

In [ ]:
!cp /content/gdrive/MyDrive/ColabNotebooks/CatDetector/tito1.jpg ./
!cp /content/gdrive/MyDrive/ColabNotebooks/CatDetector/tito2.jpg ./
!cp /content/gdrive/MyDrive/ColabNotebooks/CatDetector/tito3.jpg ./


In [ ]:
# usa a base de dados coco.names para a identificacao da classe
#!./darknet detect cfg/yolov4_custom.cfg yolov4_custom_last.weights tito3.jpg

In [ ]:
# usa a base de dados que voce criou
!./darknet detector test data/obj.data cfg/yolov4_custom.cfg yolov4_custom_last.weights tito1.jpg

In [ ]:
#funcao que facilita a deteccao
detectar("/content/darknet/tito1.jpg")